In [1]:
from datetime import datetime, timedelta

import pandas as pd
import pytz
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy import func, select
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

local_tz = pytz.timezone("Asia/Dubai")

In [2]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
date_fr = datetime.now(local_tz) - timedelta(days=30)

In [4]:
reservation_data = pd.read_sql(
    sql=select(reservation).where(
        func.date(db.gwg_reservation.c.in_date) >= date_fr.date()
    ),
    con=db.engine.connect(),
)

In [5]:
date_to = reservation_data["in_date"].max()
print(date_fr, date_to)

2024-01-22 00:19:35.432765+04:00 2025-04-24


In [6]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [7]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 1).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [8]:
gwg_data = pd.concat(dfs, axis=0, ignore_index=True)

In [9]:
reservation_data.shape

(52595, 36)

In [10]:
gwg_data.shape

(52595, 36)

In [11]:
rows_not_in_gwg = reservation_data[~reservation_data["ref_id"].isin(gwg_data["ref_id"])]
rows_not_in_gwg.empty

In [17]:
rows_not_in_db = gwg_data[~gwg_data["ref_id"].isin(reservation_data["ref_id"])]
rows_not_in_db.empty

True